In [2]:
# load artifacts (from disk)
import json, pandas as pd
from chessrl.eval.helpers import (
    vi_move_from_policy_map, vi_move_from_values,
    mcts_move_from_instance,
)
from chessrl.eval.evaluator import evaluate
from chessrl.algorithms.mcts import MCTS
from chessrl.eval.helpers import optimal_moves_syzygy

# load artifacts (from disk)
from chessrl.utils.io import load_vf_parquet, load_policy_jsonl

2025-08-25 00:55:23,167 - INFO - Loading config file...


In [3]:
TB_PATH = "../../../syzygy-tables/"

# dtm oracle
dtz_oracle = pd.read_csv(TB_PATH + "/krk_dtz.csv").set_index("fen")["dtz"].to_dict()
dtm_oracle = lambda fen: abs(int(dtz_oracle[fen]))

# optimal moves source
opt_moves   = optimal_moves_syzygy(TB_PATH)

# evaluate (same function for all)
test_fens = [ "8/8/8/8/8/8/k7/2K4R w - - 0 1", "8/8/8/8/8/8/k7/2K2R2 w - - 0 1", "8/8/8/8/8/8/k7/2K4R w - - 0 1" ] 

# MCTS

In [3]:
# 3) MCTS
mcts = MCTS(iterations=10000, seconds=0.0)
move_fn = mcts_move_from_instance(mcts, "seconds")
df_mcts   = evaluate(move_fn, test_fens, TB_PATH, dtm_oracle, optimal_moves=opt_moves, budget=6.0) # budget is in seconds
df_mcts

KeyboardInterrupt: 

# Value Iteration

## from policy

In [4]:
policy_map = load_policy_jsonl("../../../artifacts/policies/vi_krk_greedy.jsonl")

#simple mate in one endgame
test_fens = ["k7/2R5/1K6/8/8/8/8/8 w - - 0 1"]

# print(policy_map[test_fens[0]])
# print(opt_moves(test_fens[0]))

vi_agent = vi_move_from_policy_map(policy_map)
df_vi     = evaluate(vi_agent,   test_fens, TB_PATH, dtm_oracle, optimal_moves=opt_moves)
df_vi

,fen,dtm_policy,dtm_oracle,gap,success,ms_per_move,top1,top1_decisions,budget
0,k7/2R5/1K6/8/8/8/8/8 w - - 0 1,1,1,0,1,0.002802,1.0,1,None


## from value function

In [3]:
V = load_vf_parquet("../../../artifacts/values/vi_krk_values.parquet")

# V[test_fens[0]]

vi_v_agent = vi_move_from_values(V, TB_PATH)

df_vi_v   = evaluate(vi_v_agent, test_fens, TB_PATH, dtm_oracle, optimal_moves=opt_moves)
df_vi_v

,fen,dtm_policy,dtm_oracle,gap,success,ms_per_move,top1,top1_decisions,budget
0,8/8/8/8/8/8/k7/2K4R w - - 0 1,3,3,0,1,33.787917,1.0,2,None
1,8/8/8/8/8/8/k7/2K2R2 w - - 0 1,3,3,0,1,23.059252,1.0,2,None
2,8/8/8/8/8/8/k7/2K4R w - - 0 1,3,3,0,1,22.180106,1.0,2,None
